In [1]:
from z3 import *
from itertools import combinations
from typing import Sequence

Read instance file:

In [2]:
input_filename = '../../Instances/15x15.txt'

w, h, n, DX, DY = None, None, None, None, None

with open(input_filename, 'r') as f_in:
    lines = f_in.read().splitlines()

    split = lines[0].split(' ')
    w = int(split[0])
    h = int(split[1])

    n = int(lines[1])

    DX = []
    DY = []

    for i in range(int(n)):
        split = lines[i + 2].split(' ')
        DX.append(int(split[0]))
        DY.append(int(split[1]))

Solver:

In [3]:
solver = Solver()

Model:

In [4]:
XY = [(Int(f'XY_{i}_0'), Int(f'XY_{i}_1')) for i in range(n)]
R = [Bool(f'R_{i}') for i in range(n)]  # auxiliary array to keep track of rotation

TRUE_DX = [If(And(DX[i] != DY[i], R[i]), DY[i], DX[i]) for i in range(n)]  # actual X dimension
TRUE_DY = [If(And(DX[i] != DY[i], R[i]), DX[i], DY[i]) for i in range(n)]  # actual Y dimension

Constraints:

In [5]:
# Non-overlapping constraint
for (i, j) in combinations(range(n), 2):
    solver.add(Or(XY[i][0] + TRUE_DX[i] <= XY[j][0], 
                  XY[j][0] + TRUE_DX[j] <= XY[i][0],
                  XY[i][1] + TRUE_DY[i] <= XY[j][1],
                  XY[j][1] + TRUE_DY[j] <= XY[i][1]))

In [6]:
# Boundaries consistency constraint
for i in range(n):
    solver.add(XY[i][0] >=0)
    solver.add(XY[i][1] >= 0)
    solver.add(XY[i][0] + TRUE_DX[i] <= w)
    solver.add(XY[i][1] + TRUE_DY[i] <= h)

In [7]:
# Cumulative constraint
def cumulative(solver, S: Sequence, D: Sequence, R: Sequence, C: int):
    # Iterate over the durations
    for u in D:
        solver.add(
            Sum(
                [If(And(S[i] <= u, u < S[i] + D[i]), R[i], 0) for i in range(n)]
            ) <= C)

# Implied constraints
cumulative(solver,
           S=list(map(lambda t: t[0], XY)),  # take x coordinates
           D=TRUE_DX,
           R=TRUE_DY,
           C=h)
cumulative(solver,
           S=list(map(lambda t: t[1], XY)),  # take y coordinates
           D=TRUE_DY,
           R=TRUE_DX,
           C=w)

In [8]:
solver.add(R[4])

In [9]:
%%time
solver.check()

CPU times: user 2.21 s, sys: 0 ns, total: 2.21 s
Wall time: 2.22 s


sat

From Z3 model solution to file:

In [10]:
model = solver.model()

xy = [(model[XY[i][0]], model[XY[i][1]]) for i in range(n)]
r = [model[R[i]] for i in range(n)]

In [11]:
output_filename = '../../pwp_utilities/15x15-rot.txt'
with open(output_filename, 'w') as f_out:
    f_out.write('{} {}\n'.format(w, h))
    f_out.write('{}\n'.format(n))
    for i in range(n):
        f_out.write('{} {}\t{} {}\n'.format(DY[i] if r[i] else DX[i], DX[i] if r[i] else DY[i], xy[i][0], xy[i][1]))